In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pyperclip
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import requests as req
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [9]:
  # 웹드라이버 실행
driver = webdriver.Chrome()

# 제품 목록 페이지 로드
driver.get("https://www.flipkart.com/televisions/pr?sid=ckf%2Cczl&p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock&otracker=categorytree&p%5B%5D=facets.serviceability%5B%5D%3Dtrue&p%5B%5D=facets.brand%255B%255D%3DLG&otracker=nmenu_sub_TVs%20%26%20Appliances_0_LG")

# 페이지 로딩 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'CGtC98')))

# URL을 저장할 리스트 초기화
url_list = []

# 요소 찾기
elements = driver.find_elements(By.CLASS_NAME, 'CGtC98')

# 각 요소에서 'href' 속성을 가져와 리스트에 추가 (sponsored 요소는 제외)
for element in elements:
    try:
        # 'f8qK5m' 클래스를 가진 자식 요소가 있는지 확인
        if element.find_elements(By.CLASS_NAME, 'f8qK5m'):
            # 'f8qK5m' 클래스를 가진 자식 요소가 있다면 skip
            continue
        # 자식 요소가 'f8qK5m' 클래스를 가지고 있지 않다면 href 추출
        url_value = element.get_attribute('href')
        if url_value:  # href 속성이 있는 경우만 추가
            url_list.append(url_value)
    except NoSuchElementException:
        # 자식 요소가 없으면 그냥 넘어가기
        continue 
        
# 리뷰 데이터를 저장할 리스트 초기화
all_reviews = []

for url in tqdm(url_list):
    try:
        driver.get(url)
        
        # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용...
        try:
            # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용... 
            list_items = driver.find_elements(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span')
            
            # 텍스트 추출
            hp_data = [item.text for item in list_items]
            if not hp_data:
                hp_data = ["No data"]  # 데이터가 없을 경우 기본값
        except (NoSuchElementException, TimeoutException) as e:
            hp_data = ["Error fetching data"]  # 오류 발생 시 기본값
        
        # 리뷰 버튼 클릭 대기
        try:
            review_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_23J90q.RcXBOT")))
            if review_button.is_displayed():
                review_button.click()
        except (NoSuchElementException, TimeoutException):
            pass
        
        # 낮은 별점 접근
        try:
            low_rating_option = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#container > div > div.VLIitu > div > div > div.DOjaWF.gdgoEp.col-9-12 > div.DOjaWF.gdgoEp.col-12-12 > div > div:nth-child(2) > div > div > div > div > select > option:nth-child(4)")))
            low_rating_option.click()
        except (NoSuchElementException, TimeoutException) as e:
            print(f"Error selecting sort option for URL {url}: {e}")
        
        # 데이터 확보
        page_number = 1
        
        while True:
            try:
                # 페이지 URL 업데이트
                current_url = driver.current_url
                full_url = f'{current_url}&page={page_number}'
                driver.get(full_url)
                
                # ZmyHeo 클래스의 존재 여부를 확인
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ZmyHeo')))
                
                # 데이터 추출
                text_elements = driver.find_elements(By.CLASS_NAME, 'ZmyHeo')
                state_elements = driver.find_elements(By.CLASS_NAME, 'MztJPv')
                date_elements = driver.find_elements(By.CLASS_NAME, '_2NsDsF')
                star_elements = driver.find_elements(By.XPATH, "//*[contains(@class, 'XQDdHH') and (contains(@class, 'Js30Fc') or contains(@class, 'Czs3gR') or contains(@class, 'Ga3i8K'))]")
                
                text_list = [elem.text for elem in text_elements]
                state_list = [elem.text for elem in state_elements]
                date_list = [elem.text for elem in date_elements]
                dates = date_list[1::2]
                star_list = [elem.text for elem in star_elements]
                
                # 리뷰 데이터를 모든 리뷰 리스트에 추가
                for t, s, d, r in zip(text_list, state_list, dates, star_list):
                    for data in hp_data:
                        all_reviews.append({
                            'text': t,
                            'state': s,
                            'date': d,
                            'star': r,
                            'hp_data': data  # 각각의 hp_data 항목을 추가
                        })

                # 페이지 번호 증가
                page_number += 1
                
            except (NoSuchElementException, TimeoutException) as e:
                print(f"Error on page {page_number} for URL {full_url}: {e}")
                break
    
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# 현재 창 닫기
driver.quit()


  7%|█████▉                                                                             | 1/14 [00:43<09:25, 43.48s/it]

Error processing URL https://www.flipkart.com/lg-80-cm-32-inch-hd-ready-led-smart-webos-tv/p/itmb2ff267644902?pid=TVSGC8FKBCFFBFT7&lid=LSTTVSGC8FKBCFFBFT78ATBNW&marketplace=FLIPKART&store=ckf%2Fczl&spotlightTagId=BestsellerId_ckf%2Fczl&srno=b_1_3&otracker=browse&iid=5eb306d6-a4b7-4c47-b9e3-1628db20fd4f.TVSGC8FKBCFFBFT7.SEARCH&ssid=oig86eolds0000001724715592202: name 'NoSuchElementException' is not defined


 14%|███████████▊                                                                       | 2/14 [01:24<08:26, 42.25s/it]

Error processing URL https://www.flipkart.com/lg-80-cm-32-inch-hd-ready-led-smart-webos-tv-alpha75-ai-processor-gen5-refresh-rate-60-hertz-magic-remote-capability/p/itme87a122f5e1e5?pid=TVSGS8GYDQXWP8QX&lid=LSTTVSGS8GYDQXWP8QX2GXHQP&marketplace=FLIPKART&store=ckf%2Fczl&spotlightTagId=FkPickId_ckf%2Fczl&srno=b_1_4&otracker=browse&iid=5eb306d6-a4b7-4c47-b9e3-1628db20fd4f.TVSGS8GYDQXWP8QX.SEARCH&ssid=oig86eolds0000001724715592202: name 'NoSuchElementException' is not defined


 21%|█████████████████▊                                                                 | 3/14 [01:59<07:05, 38.70s/it]

...

Error processing URL https://www.flipkart.com/lg-nanocell-189-cm-75-inch-ultra-hd-4k-led-smart-webos-tv/p/itm9199805022b45?pid=TVSFHBVZ6FSMSZFZ&lid=LSTTVSFHBVZ6FSMSZFZI7ZRJU&marketplace=FLIPKART&store=ckf%2Fczl&srno=b_1_20&otracker=browse&iid=5eb306d6-a4b7-4c47-b9e3-1628db20fd4f.TVSFHBVZ6FSMSZFZ.SEARCH&ssid=oig86eolds0000001724715592202: name 'NoSuchElementException' is not defined


 93%|████████████████████████████████████████████████████████████████████████████▏     | 13/14 [06:51<00:25, 25.93s/it]

Error processing URL https://www.flipkart.com/lg-uq8020-108-cm-43-inch-ultra-hd-4k-led-smart-webos-tv-thinq-ai-filmmaker-mode-magic-remote-control/p/itm0bd784c52288a?pid=TVSGFGYSTDGT5ZWK&lid=LSTTVSGFGYSTDGT5ZWKI0BONW&marketplace=FLIPKART&store=ckf%2Fczl&srno=b_1_23&otracker=browse&iid=5eb306d6-a4b7-4c47-b9e3-1628db20fd4f.TVSGFGYSTDGT5ZWK.SEARCH&ssid=oig86eolds0000001724715592202: name 'NoSuchElementException' is not defined


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [07:01<00:00, 30.10s/it]

Error processing URL https://www.flipkart.com/lg-oled-b2-164-cm-65-inch-ultra-hd-4k-smart-webos-tv-alpha77-gen5-ai-processor-picture-pro-pixel-dimming-dolby-vision-iq-atmos-eye-comfort-display-nvidia-g-sync-vrr-amd-freesync-120-hz-refresh-rate-game-optimizer-magic-remote-control-oled65b2psa-5-years-os-upgrade/p/itm381189369f9a4?pid=TVSGR9AFUKVTNZKR&lid=LSTTVSGR9AFUKVTNZKRB1FKQP&marketplace=FLIPKART&store=ckf%2Fczl&srno=b_1_24&otracker=browse&iid=5eb306d6-a4b7-4c47-b9e3-1628db20fd4f.TVSGR9AFUKVTNZKR.SEARCH&ssid=oig86eolds0000001724715592202: name 'NoSuchElementException' is not defined


In [3]:
pd.set_option('display.max_rows', None)  # 모든 행 보기


In [11]:
df_LG_TV = pd.DataFrame(all_reviews)
df_LG_TV

,text,state,date,star,hp_data
0,Picture quality is very bad.. bright ness is m...,"Certified Buyer, Bangalore",9 months ago,1,LG 80 cm (32 inch) HD Ready LED Smart WebOS TV...
1,"Within 15 days we got a problem, small black d...","Certified Buyer, Mangalagiri",5 days ago,1,LG 80 cm (32 inch) HD Ready LED Smart WebOS TV...
2,"Very bad, stand is not flexible, stand is not ...","Certified Buyer, Jewar",8 days ago,1,LG 80 cm (32 inch) HD Ready LED Smart WebOS TV...
3,Still we are waiting for installation but the ...,"Certified Buyer, Nizamabad District",27 days ago,1,LG 80 cm (32 inch) HD Ready LED Smart WebOS TV...
4,Worst brand,"Certified Buyer, Bhiwandi",29 days ago,1,LG 80 cm (32 inch) HD Ready LED Smart WebOS TV...
5,Not a value for money even the flipkart delive...,"Certified Buyer, Vasai Virar",30 days ago,1,LG 80 cm (32 inch) HD Ready LED Smart WebOS TV...
6,Too much laggy product.,"Certified Buyer, Jamui",1 month ago,1,LG 80 cm (32 inch) HD Ready LED Smart WebOS TV...
7,"Not working lg tv,","Certified Buyer, Bengaluru",1 month ago,1,LG 80 cm (32 inch) HD Ready LED Smart WebOS TV...
8,Worst lg tv so lacking and taking time to chan...,"Certified Buyer, Visakhapatnam",1 month ago,1,LG 80 cm (32 inch) HD Ready LED Smart WebOS TV...
9,VERY BAD PRODUCT AND VERY WORST CUSTOMER CARE ...,"Certified Buyer, Bengaluru",2 months ago,1,LG 80 cm (32 inch) HD Ready LED Smart WebOS TV...


In [12]:
df_LG_TV.to_csv("df_LG_TV.csv")